## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-15-03-51 +0000,nypost,16-year-old Tesla driver killed in horror plun...,https://nypost.com/2026/02/06/us-news/16-year-...
1,2026-02-06-15-03-00 +0000,wsj,Japanese Yen Recovery Requires Reduced Fiscal ...,https://www.wsj.com/finance/currencies/yen-str...
2,2026-02-06-15-02-52 +0000,nyt,"Norway Rallies Behind Royals, Despite Dismay O...",https://www.nytimes.com/2026/02/06/world/europ...
3,2026-02-06-14-59-51 +0000,bbc,Amazon shares fall as it joins Big Tech AI spe...,https://www.bbc.com/news/articles/c150e144we3o...
4,2026-02-06-14-58-00 +0000,wsj,Iran Refuses to End Nuclear Enrichment in Talk...,https://www.wsj.com/world/middle-east/u-s-and-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2414/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
48,trump,31
20,epstein,21
35,talks,15
191,new,15
30,iran,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
253,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...,92
204,2026-02-06-01-19-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,77
262,2026-02-05-22-22-03 +0000,nypost,Hillary Clinton demands public hearing on Epst...,https://nypost.com/2026/02/05/us-news/hillary-...,74
229,2026-02-05-23-46-58 +0000,nyt,Bitcoin Drops to Lowest Price Since Trump Was ...,https://www.nytimes.com/2026/02/05/technology/...,67
104,2026-02-06-11-30-00 +0000,wsj,Trump’s Tax Law Sharply Cuts Amazon’s Corporat...,https://www.wsj.com/business/trumps-tax-law-sh...,64


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
253,92,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...
54,58,2026-02-06-13-31-30 +0000,wapo,Top Russian general shot in Moscow as talks st...,https://www.washingtonpost.com/world/2026/02/0...
186,55,2026-02-06-04-26-26 +0000,nypost,AOC touts money for her NYC district in fundin...,https://nypost.com/2026/02/05/us-news/aoc-tout...
18,53,2026-02-06-14-36-27 +0000,nypost,Newly released FBI logs raise questions over m...,https://nypost.com/2026/02/06/us-news/newly-re...
154,46,2026-02-06-09-26-57 +0000,nyt,Milan Winter Olympics Opening Ceremony: What T...,https://www.nytimes.com/2026/02/06/world/europ...
3,46,2026-02-06-14-59-51 +0000,bbc,Amazon shares fall as it joins Big Tech AI spe...,https://www.bbc.com/news/articles/c150e144we3o...
71,43,2026-02-06-12-45-31 +0000,wapo,U.S. and Iran in Oman to discuss Iran’s nuclea...,https://www.washingtonpost.com/national-securi...
15,43,2026-02-06-14-42-50 +0000,nyt,Savannah Guthrie’s Brother Renews Plea for Ret...,https://www.nytimes.com/2026/02/06/us/savannah...
229,36,2026-02-05-23-46-58 +0000,nyt,Bitcoin Drops to Lowest Price Since Trump Was ...,https://www.nytimes.com/2026/02/05/technology/...
304,35,2026-02-05-20-15-52 +0000,bbc,Bad Bunny says he's losing sleep over the Supe...,https://www.bbc.com/news/articles/clyzwd2n75no...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
